<a href="https://colab.research.google.com/github/Georgemburu/MACHINE-LEARNING/blob/master/Classify_Structurd_Data_With_Feature_Columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# THE DATASET
%tensorflow_version 2.x
!pip install -q sklearn

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division,print_function,unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [4]:
# Split the dataframe into train, validation, and test
train,test = train_test_split(dataframe,test_size=0.2)
train,val = train_test_split(train,test_size=0.2)
print(len(train),'train examples')
print(len(val), 'validation examples')
print(len(test),'test examples')

193 train examples
49 validation examples
61 test examples


In [0]:
# Create an input pipeline
#############################
def df_to_dataset(dataframe,shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 5 
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val,shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test,shuffle=False, batch_size=batch_size)

In [9]:
# Understand the input pipeline

for feature_batch,label_batch in train_ds.take(1):
  print('Every feature:',list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:',label_batch)

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([49 54 54 47 61], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 0 1 0 0], shape=(5,), dtype=int32)


In [18]:
# Demonstrate several types of feature column
############################################
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]
example_batch

{'age': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([51, 63, 61, 49, 64], dtype=int32)>,
 'ca': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 2, 0, 3, 1], dtype=int32)>,
 'chol': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([295, 269, 307, 188, 246], dtype=int32)>,
 'cp': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([3, 4, 4, 3, 4], dtype=int32)>,
 'exang': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 1, 0, 1], dtype=int32)>,
 'fbs': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 0], dtype=int32)>,
 'oldpeak': <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.6, 1.8, 1. , 2. , 2.2])>,
 'restecg': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([2, 0, 2, 0, 2], dtype=int32)>,
 'sex': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 0, 0, 1, 1], dtype=int32)>,
 'slope': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 2, 2, 2, 3], dtype=int32)>,
 'thal': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'normal', b'normal', b

In [0]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [23]:
# 1. NUMERICAL COLUMNS
age = feature_column.numeric_column('age')
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[51.]
 [63.]
 [61.]
 [49.]
 [64.]]


NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [24]:
# 2. BUCKETIZED COLUMNS
age_buckets = feature_column.bucketized_column(
    age,
    boundaries=[18,25,30,35,40,45,50,55,60,65])
demo(age_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [26]:
# 3. CATEGORICAL COLUMNS
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal',['fixed','normal','reversible']
)

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [28]:
# 4. EMBEDDING COLUMNS
####################
# notice the input to the embedding column is the categorical
# column we previously created
thal_embedding = feature_column.embedding_column(thal,dimension=8)
demo(thal_embedding)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[-0.29739302  0.04319477 -0.431109   -0.00847093  0.00256794 -0.27585518
  -0.08013561 -0.2949652 ]
 [-0.29739302  0.04319477 -0.431109   -0.00847093  0.00256794 -0.27585518
  -0.08013561 -0.2949652 ]
 [ 0.00367211 -0.4001056  -0.37490988  0.07836268 -0.20575327  0.25015047
   0.03013948 -0.16496688]
 [ 0.00367211 -0.4001056  -0.37490988  0.07836268 -0.20575327  0.25015047
   0.03013948 -0.16496688]
 [-0.29739302  0.04319477 -0.431109   -0.00847093  0.00256794 -0.27585518
  -0.08013561 -0.2949652 ]]


In [29]:
# 5. HASHED FEATURE COLUMNS
#########################
thal_hashed = feature_column.categorical_column_with_hash_bucket(
    'thal',hash_bucket_size=1000
)
demo(feature_column.indicator_column(thal_hashed))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
# 6. CROSSED FEATURE COLUMNS
###########################
crossed_feature = feature_column.crossed_column([age_buckets,thal],hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
# CHOOSE WHICH COLUMNS TO USE
#############################

feature_columns = []

# numeric cols
for header in ['age','trestbps','chol','thalach','oldpeak','slope','ca']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(
    age, boundaries=[18,25,30,35,40,45,50,55,60,65]
)
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal',['fixed','normal','reversible']
)
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal,dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets,thal],
                                                hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [33]:
feature_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='trestbps', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='chol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='thalach', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='oldpeak', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='slope', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ca', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 BucketizedColumn(source_column=NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(18, 25, 30, 35, 40, 45, 50, 55, 60, 65)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='thal', vocabulary_list=('fixed', 'normal', 'reversi

In [0]:
# CREATE A FEATURE LAYER
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
# Create a new input pipeline
batch_size = 32
train_ds = df_to_dataset(train,batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test,shuffle=False,batch_size=batch_size)

In [38]:
# CREATE, COMPILE AND TRAIN THE MODEL
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Train for 7 steps, validate for 2 steps
Epoch 1/10
7/7 [==============================] - 1s 178ms/step - loss: 1.5886 - accuracy: 0.5078 - val_loss: 1.2287 - val_accuracy: 0.6122
Epoch 2/10
7/7 [==============================] - 0s 11ms/step - loss: 2.6628 - accuracy: 0.7254 - val_loss: 1.5667 - val_accuracy: 0.6122
Epoch 3/10
7/7 [==============================] - 0s 10ms/step - loss: 1.1394 - accuracy: 0.5026 - val_loss: 0.6053 - val_accuracy: 0.7143
Epoch 4/10
7/7 [==============================] - 0s 10ms/step - loss: 1.3033 - accuracy: 0.7306 - val_loss: 1.0843 - val_accuracy: 0.6122
Epoch 5/10
7/7 [==============================] - 0s 10ms/step - loss: 0.7434 - accuracy: 0.6321 - val_loss: 0.6218 - val_accuracy: 0.6939
Epoch 6/10
7/7 [==============================] - 0s 11ms/step - loss: 0.6081 - accuracy: 0.6943 - val_loss: 0.9440 - val_accuracy: 0.6122
Epoch 7/10
7/7 [==============================] - 0s 11ms/step - loss: 0.5748 - accuracy: 0.7409 - val_loss: 0.6043 - val_acc

In [39]:
loss, accuracy = model.evaluate(test_ds)
print('Accuracy',accuracy)

2/2 [==============================] - 0s 4ms/step - loss: 0.3642 - accuracy: 0.8525
Accuracy 0.852459
